<a href="https://colab.research.google.com/github/neoxlarge/GoldenCross/blob/%E5%8F%B0%E8%82%A1/vegas_ema_v0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



>
1.   請自行儲存復本修改
2.   按"執行階段"/"全部執行"
3.   或者每格順序執行
4.   ema,timeframe參數可自行修改
5.   抓取FTX全部USD交易對,再檢查在設定的timeframe是否出現黃金交叉,再畫出線圖.
6.   可以自行修改放到Deepnote跑telegram bot,bot token 和　chat id,請自行google. 並把最後一行#send_to_bot_photo..., 的#號刪掉就可以跑telegram bot了.
7.    測試用的telegram channel: https://t.me/neoxbot_test       





In [ ]:
#@title User Setting { display-mode: "both" }

#ema setting
ema_s = 12 #@param {type:"integer"}
ema_m1 = 55 #@param {type:"integer"}
ema_m2 = 76 #@param {type:"integer"}
ema_h1 = 144 #@param {type:"integer"}
ema_h2 = 169 #@param {type:"integer"}
sma_s = 20 #@param {type:"integer"}


#ftx supprted timeframe ['15s', '1m', '5m', '15m', '1h', '4h', '1d', '3d', '1w', '2w', '1M']
#timeframe
t_frame = "1h" #@param ['1h', '4h', '1d', '3d']

#線圖的佈景 https://plotly.com/python/templates/
plotly_theme = "plotly_dark" #@param ["plotly_dark", "plotly_white", "plotly", "ggplot2", "seaborn", "simple_white", "none"]

#@markdown Telegram bot setting
bot_token = '' #@param {type:"string"}
chat_id = '' #@param {type:"string"}


Deepnote 無法安裝talib-binance, 需要特別的安裝方式.參考[用 Python 快速計算 158 種技術指標](https://medium.com/ai%E8%82%A1%E4%BB%94/%E7%94%A8-python-%E5%BF%AB%E9%80%9F%E8%A8%88%E7%AE%97-158-%E7%A8%AE%E6%8A%80%E8%A1%93%E6%8C%87%E6%A8%99-26f9579b8f3a)



In [ ]:
!rm -f *.png

#安裝套件
import os

!pip install jedi

try:
    import kaleido
except ModuleNotFoundError as e:
    !pip install -U kaleido    

try:
    import talib
except ModuleNotFoundError as e:
    if os.path.exists("./talib"):
        %cd ta-lib
    else:    
        !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
        !tar -xzvf ta-lib-0.4.0-src.tar.gz
        %cd ta-lib
        !./configure --prefix=/usr
        !make
        !make install
    !pip install Ta-Lib
    %cd ..
    !rm -f ta-lib-0.4.0-src.tar.gz   
    #!rm -rf ta-lib   

try:
    import ccxt
except ModuleNotFoundError as e:
    !pip install ccxt                    
!exit

In [ ]:
#載入模組

import talib
import requests
import pandas as pd
import numpy as np
import datetime as dt
import time
import ccxt
import plotly.graph_objects as px


In [ ]:
#crypto
#獲取FTX的現貨/USD交易對
exchange = ccxt.ftx()
exchange.load_markets()
symbol_list = set()
ex_symbol_list = set()
#排除底下字串
ex_string=("USD:USD","BULL","BEAR","HALF","HEDGE","JPY","EUR","TRYB","AUD","/BTC","/DOGE","/USDT","GBP","HKD","ZAR","CAD","BRZ","MXN","ARS","DAI")
for i in exchange.symbols:
    for j in ex_string:
        if i.find(j) != -1:
            ex_symbol_list.add(i)

crypto_symbol_list = set(exchange.symbols) - ex_symbol_list #set差集運算

#print(len(symbol_list))
#print(symbol_list)
     

In [ ]:
#crypto
#自定義交易對列表 2022/08/26
#如果要使用自定的交易對,請把symbol_inFTX 改成 symbol_list.
crypto_symbol_inFTX = ['1INCH/USD', 'AAPL/USD', 'AAVE/USD', 'ABNB/USD', 'ACB/USD', 'AGLD/USD', 'AKRO/USD', 'ALCX/USD',
        'ALEPH/USD', 'ALGO/USD', 'ALICE/USD', 'ALPHA/USD', 'AMC Entertainment Holdings/USD',
        'AMD/USD', 'AMPL/USD', 'AMZN/USD', 'ANC/USD', 'APE/USD', 'APHA/USD', 'ARKK/USD',
        'ASD/USD', 'ATLAS/USD', 'ATOM/USD', 'AURY/USD', 'AVAX/USD', 'AXS/USD', 'BABA/USD',
        'BADGER/USD', 'BAL/USD', 'BAND/USD', 'BAO/USD', 'BAR/USD', 'BAT/USD', 'BB/USD',
        'BCH/USD', 'BICO/USD', 'BILI/USD', 'BIT/USD', 'BITO/USD', 'BITW/USD', 'BLT/USD',
        'BNB/USD', 'BNT/USD', 'BNTX/USD', 'BOBA/USD', 'BTC/USD', 'BTT/USD', 'BVOL/USD',
        'BYND/USD', 'C98/USD', 'CEL/USD', 'CGC/USD', 'CHR/USD', 'CHZ/USD', 'CITY/USD',
        'CLV/USD', 'COIN/USD', 'COMP/USD', 'CONV/USD', 'COPE/USD', 'CQT/USD', 'CREAM/USD',
        'CRO/USD', 'CRON/USD', 'CRV/USD', 'CTX/USD', 'CUSDT/USD', 'CVC/USD', 'CVX/USD',
        'DAWN/USD', 'DENT/USD', 'DFL/USD', 'DKNG/USD', 'DMG/USD', 'DODO/USD', 'DOGE/USD',
        'DOT/USD', 'DYDX/USD', 'EDEN/USD', 'EMB/USD', 'ENJ/USD', 'ENS/USD', 'ETH/USD',
        'ETHE/USD', 'FB/USD', 'FIDA/USD', 'FRONT/USD', 'FTM/USD', 'FTT/USD', 'FXS/USD',
        'GAL/USD', 'GALA/USD', 'GALFAN/USD', 'GARI/USD', 'GBTC/USD', 'GDX/USD', 'GDXJ/USD',
        'GENE/USD', 'GLD/USD', 'GLXY/USD', 'GME/USD', 'GMT/USD', 'GODS/USD', 'GOG/USD',
        'GOOGL/USD', 'GRT/USD', 'GST/USD', 'GT/USD', 'HGET/USD', 'HMT/USD', 'HNT/USD',
        'HOLY/USD', 'HOOD/USD', 'HT/USD', 'HUM/USD', 'HXRO/USD', 'IBVOL/USD', 'IMX/USD',
        'INDI/USD', 'INTER/USD', 'IP3/USD', 'JET/USD', 'JOE/USD', 'JST/USD', 'KBTT/USD',
        'KIN/USD', 'KNC/USD', 'KSHIB/USD', 'KSOS/USD', 'LDO/USD', 'LEO/USD', 'LINA/USD',
        'LINK/USD', 'LOOKS/USD', 'LRC/USD', 'LTC/USD', 'LUA/USD', 'MANA/USD', 'MAPS/USD',
        'MATH/USD', 'MATIC/USD', 'MBS/USD', 'MCB/USD', 'MEDIA/USD', 'MER/USD', 'MKR/USD',
        'MNGO/USD', 'MOB/USD', 'MRNA/USD', 'MSOL/USD', 'MSTR/USD', 'MTA/USD', 'MTL/USD',
        'NEAR/USD', 'NEXO/USD', 'NFLX/USD', 'NIO/USD', 'NOK/USD', 'NVDA/USD', 'OKB/USD',
        'OMG/USD', 'ORBS/USD', 'ORCA/USD', 'OXY/USD', 'PAXG/USD', 'PENN/USD', 'PEOPLE/USD',
        'PERP/USD', 'PFE/USD', 'POLIS/USD', 'PORT/USD', 'PRISM/USD', 'PROM/USD', 'PSG/USD',
        'PSY/USD', 'PTU/USD', 'PUNDIX/USD', 'PYPL/USD', 'QI/USD', 'RAY/USD', 'REAL/USD',
        'REEF/USD', 'REN/USD', 'RNDR/USD', 'ROOK/USD', 'RSR/USD', 'SAND/USD', 'SECO/USD',
        'SHIB/USD', 'SKL/USD', 'SLND/USD', 'SLP/USD', 'SLRS/USD', 'SLV/USD', 'SNX/USD',
        'SNY/USD', 'SOL/USD', 'SOS/USD', 'SPA/USD', 'SPELL/USD', 'SPY/USD', 'SQ/USD',
        'SRM/USD', 'STEP/USD', 'STETH/USD', 'STG/USD', 'STMX/USD', 'STORJ/USD', 'STSOL/USD',
        'SUN/USD', 'SUSHI/USD', 'SXP/USD', 'StarLaunch/USD', 'TLM/USD', 'TLRY/USD', 'TOMO/USD',
        'TONCOIN/USD', 'TRU/USD', 'TRX/USD', 'TSLA/USD', 'TSM/USD', 'TULIP/USD', 'TWTR/USD',
        'UBER/USD', 'UBXT/USD', 'UMEE/USD', 'UNI/USD', 'USDT/USD', 'USO/USD', 'VGX/USD',
        'WAVES/USD', 'WBTC/USD', 'WtFLOW/USD', 'WNDR/USD', 'WRX/USD', 'XAUT/USD', 'XRP/USD',
        'YFI/USD', 'YFII/USD', 'YGG/USD', 'ZM/USD', 'ZRX/USD']
 

In [ ]:
#tw stock
#get 股票代号
!pip install twstock

import twstock

twstock.__update_codes()

stock_list = twstock.twse
twstock_symbol_list = []

for i in stock_list:
    if stock_list[i][0] in ('ETF','股票'):
        twstock_symbol_list.append((i,stock_list[i][2])) 

plotly畫K線

telegram bot sendphoto
[telegram api sendphoto](https://core.telegram.org/bots/api#sendphoto)

[code sample](https://stackoverflow.com/questions/36778321/how-to-send-photo-on-telegram-bot)

In [ ]:
#define function
#
#get ohlc_data
def get_ohlc_data(symbol, timeframe="1h"):
    ohlc_data = exchange.fetch_ohlcv(symbol,timeframe=timeframe,limit=400)
    
    for i in ohlc_data:
        i[0] = dt.datetime.fromtimestamp(i[0]/1000)

    ohlc_df = pd.DataFrame(ohlc_data,columns=["date","open","high","low","close","volume"]) 
    
    ohlc_df.date = pd.to_datetime(ohlc_df.date)
    
    ema_sline = talib.EMA(ohlc_df["close"],timeperiod=ema_s)
    ema_m1line = talib.EMA(ohlc_df["close"],timeperiod=ema_m1)
    ema_m2line = talib.EMA(ohlc_df["close"],timeperiod=ema_m2)
    ema_h1line = talib.EMA(ohlc_df["close"],timeperiod=ema_h1)
    ema_h2line = talib.EMA(ohlc_df["close"],timeperiod=ema_h2)
    sma_sline = talib.SMA(ohlc_df["close"],timeperiod=sma_s)

    ohlc_df['ema_sline'] = ema_sline
    ohlc_df['ema_m1line'] = ema_m1line
    ohlc_df['ema_m2line'] = ema_m2line
    ohlc_df['ema_h1line'] = ema_h1line
    ohlc_df['ema_h2line'] = ema_h2line
    ohlc_df['sma_sline'] = sma_sline

    data = {
        'symbol' : symbol,
        'ohlc_df' : ohlc_df.tail(150),
        'timeframe' : timeframe
    }
    return data

#檢查黃金交差, ema_s向上交叉ema_m1或ema_m2,選最高者
def check_long_condition_ema(data):
    long_condition = False
    if data["ohlc_df"]["ema_m1line"].iat[-1] > data["ohlc_df"]["ema_m2line"].iat[-1]:
        if data["ohlc_df"]["ema_sline"].iat[-1] > data["ohlc_df"]["ema_m1line"].iat[-1] \
        and data["ohlc_df"]["ema_sline"].iat[-2] < data["ohlc_df"]["ema_m1line"].iat[-2]:
            long_condition = True
    else:
        if data["ohlc_df"]["ema_sline"].iat[-1] > data["ohlc_df"]["ema_m2line"].iat[-1] \
        and data["ohlc_df"]["ema_sline"].iat[-2] < data["ohlc_df"]["ema_m2line"].iat[-2]:
            long_condition = True

    return long_condition     

#檢查死亡交差, ema_s向下交叉ema_m1或ema_m2,選最低者
def check_short_condition_ema(data):
    short_condition = False
    
    if data["ohlc_df"]["ema_m1line"].iat[-1] < data["ohlc_df"]["ema_m2line"].iat[-1]:
        if data["ohlc_df"]["ema_sline"].iat[-1] < data["ohlc_df"]["ema_m1line"].iat[-1] \
        and data["ohlc_df"]["ema_sline"].iat[-2] > data["ohlc_df"]["ema_m1line"].iat[-2]:
            short_condition = True
    else:
        if data["ohlc_df"]["ema_sline"].iat[-1] < data["ohlc_df"]["ema_m2line"].iat[-1] \
        and data["ohlc_df"]["ema_sline"].iat[-2] > data["ohlc_df"]["ema_m2line"].iat[-2]:
            short_condition = True

    return short_condition     

#檢查黃金交差, ema_s向上交叉sma_s
def check_long_condition_sma(data):
    long_condition = False

    if data["ohlc_df"]["ema_sline"].iat[-1] > data["ohlc_df"]["sma_sline"].iat[-1] \
    and data["ohlc_df"]["ema_sline"].iat[-2] < data["ohlc_df"]["sma_sline"].iat[-2]:
        long_conditon = True

    return long_condition

#檢查死亡交差, ema_s向下交叉sma_s
def check_short_condition_sma(data):
    short_condition = False

    if data["ohlc_df"]["ema_sline"].iat[-1] < data["ohlc_df"]["sma_sline"].iat[-1] \
        and data["ohlc_df"]["ema_sline"].iat[-2] > data["ohlc_df"]["sma_sline"].iat[-2]:
        short_conditon = True

    return short_condition    

#畫K線
def plotly_kline(data):
    candlestick = px.Candlestick(x=data["ohlc_df"]["date"],
                    open=data["ohlc_df"]['open'],
                    high=data["ohlc_df"]['high'],
                    low=data["ohlc_df"]['low'],
                    close=data["ohlc_df"]['close'],name="Kline")

    ema_sline = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['ema_sline'],
                    line=dict(color='orange', width=1),
                    mode='lines',name=f"EMA{ema_s}")
    
    ema_m1line = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['ema_m1line'],
                    line=dict(color='green', width=1),
                    mode='lines',name=f"EMA{ema_m1}")

    ema_m2line = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['ema_m2line'],
                    line=dict(color='blue', width=1),
                    mode='lines',name=f"EMA{ema_m2}")

    ema_h1line = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['ema_h1line'],
                    line=dict(color='pink', width=1),
                    mode='lines',name=f"EMA{ema_h1}")

    ema_h2line = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['ema_h2line'],
                    line=dict(color='red', width=1),
                    mode='lines',name=f"EMA{ema_h2}")

    sma_sline = px.Scatter(x=data["ohlc_df"]["date"], 
                    y=data["ohlc_df"]['sma_sline'],
                    line=dict(color='yellow', width=1),
                    mode='lines',name=f"SMA{sma_s}")
    
    sma_deduct_df=data['ohlc_df'][(0-sma_s):(0-sma_s+1)]
    sma_deduct = px.Scatter(x=sma_deduct_df['date'],y=sma_deduct_df['close'],
                    mode='markers',name=f"SMA{sma_s}_deduct",
                    marker=(dict(symbol="arrow-bar-right",size=20)))

    fig = px.Figure(data=[candlestick,ema_sline,ema_m1line,ema_m2line,ema_h1line,ema_h2line,sma_sline,sma_deduct])
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_layout(title=f"{data['symbol']} timeframe:{data['timeframe']}",autosize=False,width=1000,height=400)
    fig["layout"]["template"]=plotly_theme

    fig.show()
    fig.write_image(f"{data['symbol'].split('/')[0]}.png")

def send_to_bot_msg(tid_, msg, bot_token_):
    url = f'https://api.telegram.org/bot{bot_token_}/sendMessage'
    headers = {'content-type': 'application/json'}
    data  = f'{{"chat_id": "{tid_}", "text": "{msg}", "parse_mode": "HTML", "disable_web_page_preview": "true"}}'.encode('utf-8')
    r = requests.post(url, data=data, headers=headers, verify=False)  

def send_to_bot_photo(tid_, msg, bot_token_, photo_opened):
    url = f'https://api.telegram.org/bot{bot_token_}/sendPhoto'
    headers = {'content-type':'multipart/form-data'}
    data  = {"chat_id": tid_, "caption": msg, "parse_mode": "HTML", "disable_web_page_preview": "true"}
    photo_data = {'photo': photo_opened}
    r = requests.post(url, data=data, files=photo_data) 
    


# 如何建立telegram bot頻道
1.找BotFather建立機器人,取得token. 請參考[Day 16-開始製作Telegram聊天機器人！](https://ithelp.ithome.com.tw/articles/10244411)

2.建立一個頻道,並把機器人加進去,取得chat_id.每一個對話, 群組, 頻道都是一個chat_id, 請參考[取得chat_id](http://blog.3dgowl.com/telegram-telegram%E4%BA%94-%E5%8F%96%E5%BE%97-chat-id/)

3.把token 和chat_id 放到底下程式碼中. 再把最後一行開頭的#號刪除
`send_to_bot_photo(tid_=chat_id, msg=msg, bot_token_=bot_token, photo_opened=open(f"{symbol_name}.png", "rb"))`

4.下載這份程式為*.ipynb檔案.

5.到deepnote.com建立帳號,新增一個project ,把*,ipynb上傳. 

6.按Run notebook可以手動執行, 按右方小鍵頭可以排程.timeframe 1h就每小時跑，1h以上就依你的時框去排程, 如果timeframe 1d 排每小時跑, 應該會一直收到重覆訊息.


# 如何改成ema向上交叉sma
請把這兩行的#號掉換,#號是注釋,不會執行
```
    #is_long = check_long_condition_ema(ema_data)
    is_long = check_long_condition_sma(ema_data)
```




In [ ]:
#start
for symbol in symbol_list:
    ema_data = get_ohlc_data(symbol,timeframe=t_frame)

    is_long = check_long_condition_ema(ema_data)
    #is_long = check_long_condition_sma(ema_data)

    is_short = check_short_condition_ema(ema_data)
    #is_short = check_short_condition_sma(ema_data)


    symbol_name = symbol.split("/")[0]

    if is_long == True:
        msg = f"💰💰💰 {symbol} is Golden Cross!!"
        print(msg)
        plotly_kline(ema_data)    
        #send photo
        send_to_bot_photo(tid_=chat_id, msg=msg, bot_token_=bot_token, photo_opened=open(f"{symbol_name}.png", "rb"))


    elif is_short == True:
        msg = f"☠️☠️☠️ {symbol} is Dead Cross!!"
        print(msg)
        plotly_kline(ema_data)    
        #send photo
        send_to_bot_photo(tid_=chat_id, msg=msg, bot_token_=bot_token, photo_opened=open(f"{symbol_name}.png", "rb"))